In [1]:
import pipeline
import pandas as pd
import importlib
importlib.reload(pipeline)

<module 'pipeline' from '/Users/piyushtank/Dropbox/mscapp/spring_20/machine_learning/project/covid/git/ml-covid/scripts/pipeline.py'>

In [2]:
filepath = '../data/covid_df_protocol_3.pkl'
weeks_for_cv = [0,1,2,3,4,5,6]
df = pd.read_pickle(filepath)
n_weeks_prediction = 2
results = {}
for i in weeks_for_cv:
    shorter_df = pipeline.cut_df_on_weeks(df, i)
    df_train, df_test = pipeline.split_and_scale_on_last_weeks(shorter_df, n_weeks_prediction)
    df_train = df_train.dropna()
    df_test = df_test.dropna()
    X_train, y_train = pipeline.divide_target_and_features(df_train, 'Confirmed Cases')
    X_test, y_test = pipeline.divide_target_and_features(df_test, 'Confirmed Cases')
    dct = pipeline.train_and_evaluate(X_train, y_train, X_test, y_test)
    print(dct)
    results['Cutting weeks: ' +str(i)] = dct


Cutting dataframe on date: 2020-05-29


/Users/piyushtank/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 14394669572029.89, tolerance: 4800287932.3951435
  positive)


Lasso: Features with highest magnitude                                coefficients in absolute value
                              Feature    Coefficient
18                    StringencyIndex -195985.573766
61                      Country_China  -76062.329867
170  Country_United States of America   62022.325921
97                      Country_Italy   57889.856221
153                     Country_Spain   50107.601763
99                      Country_Japan  -29739.463610
2                C2_Workplace closing   28081.399386
1                   C1_School closing   26319.548122
51                     Country_Brazil  -25672.144792
13    H1_Public information campaigns   25221.082677

Bias: 1465869738.52
Mean squared error: 14040528909.39
RSS: 20864225959352.12
Variance score: 0.40

R2 score: 0.39

Ridge: Features with highest magnitude                                coefficients in absolute value
                              Feature   Coefficient
18                    StringencyIndex -71293.0

/Users/piyushtank/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9928546332949.154, tolerance: 3250546680.4664173
  positive)


Lasso: Features with highest magnitude                                coefficients in absolute value
                              Feature    Coefficient
18                    StringencyIndex -165517.031077
99                      Country_Italy   51074.946855
172  Country_United States of America   48539.133226
61                      Country_China  -48201.556561
155                     Country_Spain   45387.735584
101                     Country_Japan  -25330.889171
2                C2_Workplace closing   23706.132356
51                     Country_Brazil  -22297.490174
1                   C1_School closing   21996.018593
13    H1_Public information campaigns   21458.813939

Bias: 1079170390.98
Mean squared error: 10958776094.59
RSS: 21566871354148.64
Variance score: 0.38

R2 score: 0.36

Ridge: Features with highest magnitude                                coefficients in absolute value
                              Feature   Coefficient
18                    StringencyIndex -60307.2

...

Lasso: Features with highest magnitude                                coefficients in absolute value
                            Feature   Coefficient
18                  StringencyIndex -71431.830084
98                    Country_Italy  24920.931921
154                   Country_Spain  22032.873854
100                   Country_Japan -19590.445079
49   Country_Bosnia and Herzegovina -13021.960756
51                   Country_Brazil -11598.711190
21             EconomicSupportIndex -11559.668525
94                     Country_Iran  11468.536850
2              C2_Workplace closing  10023.708940
9                 E1_Income support   9902.702837

Bias: 254919876.70
Mean squared error: 5241102267.02
RSS: 10607990988443.11
Variance score: 0.34

R2 score: 0.24

Ridge: Features with highest magnitude                                coefficients in absolute value
                            Feature   Coefficient
18                  StringencyIndex -26049.355279
20           ContainmentHealthInd

/Users/piyushtank/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 690775229907.6277, tolerance: 258586145.3974994
  positive)


Lasso: Features with highest magnitude                                coefficients in absolute value
                            Feature   Coefficient
18                  StringencyIndex -48942.929296
99                    Country_Italy  18420.513569
62                    Country_China  16519.301144
155                   Country_Spain  15455.145349
95                     Country_Iran   9199.261893
101                   Country_Japan  -8858.061841
49   Country_Bosnia and Herzegovina  -8301.110337
51                   Country_Brazil  -7680.773547
21             EconomicSupportIndex  -7058.067152
1                 C1_School closing   6942.765205

Bias: 113780384.14
Mean squared error: 3518850524.40
RSS: 7167898518193.98
Variance score: 0.35

R2 score: 0.20

Ridge: Features with highest magnitude                                coefficients in absolute value
                            Feature   Coefficient
18                  StringencyIndex -17647.436964
20           ContainmentHealthInde

/Users/piyushtank/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 189545032514.52628, tolerance: 93341291.27484822
  positive)


Lasso: Features with highest magnitude                                coefficients in absolute value
                             Feature   Coefficient
62                     Country_China  29830.826547
18                   StringencyIndex -28940.104403
100                    Country_Italy  12394.112206
156                    Country_Spain   9015.212473
96                      Country_Iran   6167.119173
13   H1_Public information campaigns   4448.774239
1                  C1_School closing   4426.738426
86                   Country_Germany   4068.367510
6       C6_Stay at home requirements   4028.374132
21              EconomicSupportIndex  -4002.693925

Bias: 38534259.88
Mean squared error: 2035523129.76
RSS: 4170786892877.92
Variance score: 0.44

R2 score: 0.15

Ridge: Features with highest magnitude                                coefficients in absolute value
                              Feature   Coefficient
62                      Country_China  14506.022889
94                  

Checking variables with highest percentage of NAs in training set:

In [46]:
metrics_r2 = pd.DataFrame(columns=['Lasso','Ridge','Linear Regression'])
for i, mkey in enumerate(results):
    for key in results[mkey]:
        metrics_r2.loc[i,key] = results[mkey][key][4]
metrics_r2

,Lasso,Ridge,Linear Regression
0,0.392167,0.391986,0.396079
1,0.358741,0.358856,0.361428
2,0.320707,0.320969,0.320998
3,0.280243,0.280612,0.2798
4,0.23999,0.240449,0.240953
5,0.195622,0.196314,0.197322
6,0.150177,0.151265,0.152435


In [45]:
metrics_mse = pd.DataFrame(columns=['Lasso','Ridge','Linear Regression'])
for i, mkey in enumerate(results):
    for key in results[mkey]:
        metrics_mse.loc[i,key] = results[mkey][key][1]
metrics_mse

,Lasso,Ridge,Linear Regression
0,1.40405e+10,1.40447e+10,1.39502e+10
1,1.09588e+10,1.09568e+10,1.09128e+10
2,8.90588e+09,8.90244e+09,8.90207e+09
3,7.12968e+09,7.12603e+09,7.13407e+09
4,5.2411e+09,5.23794e+09,5.23446e+09
5,3.51885e+09,3.51582e+09,3.51141e+09
6,2.03552e+09,2.03292e+09,2.03011e+09
